In [1]:
import os
import openai
from dotenv import load_dotenv
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
driver = webdriver.Chrome('drivers/chromedriver108')

# Global Context

In [4]:
base = 'https://demo.saleor.io/default-channel/en-US/'

In [5]:
driver.get(base)

In [6]:
doc = bs(driver.find_element(By.TAG_NAME, 'html').get_attribute('innerHTML'), 'html.parser')

In [7]:
title = str(doc.head.title) + '\n'
metas = '\n'.join(list(map(lambda x: str(x), doc.head.find_all('meta'))))
head_string = title + metas

In [8]:
category_prompt = '''The following text is the url and content of head tag of a website.
url: {0}
head: {1}
The website category is:'''.format(base, head_string)

In [9]:
category_prompt

'The following text is the url and content of head tag of a website.\nurl: https://demo.saleor.io/default-channel/en-US/\nhead: <title>Saleor React Storefront</title>\n<meta charset="utf-8"/>\n<meta content="width=device-width" name="viewport"/>\n<meta content="index,follow" name="robots"/>\n<meta content="Saleor React Storefront" property="og:title"/>\n<meta content="https://reactstorefront.vercel.app/default-channel/en-US/" property="og:url"/>\n<meta content="https://og-image.vercel.app/React%20Storefront.png?theme=light&amp;md=1&amp;fontSize=100px&amp;images=https%3A%2F%2Fassets.vercel.com%2Fimage%2Fupload%2Ffront%2Fassets%2Fdesign%2Fvercel-triangle-black.svg&amp;images=https%3A%2F%2Fsaleor.io%2Fstatic%2Flogo-ad1b99aa7c6f5acf58a61640af760cfd.svg" property="og:image"/>\n<meta content="Hero image" property="og:image:alt"/>\n<meta content="2048" property="og:image:width"/>\n<meta content="1170" property="og:image:height"/>\n<meta content="Saleor React Storefront" property="og:site_name

In [10]:
category_res = openai.Completion.create(
    model="text-davinci-003",
    prompt=category_prompt,
    temperature=0,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    max_tokens=256
)

category_res

<OpenAIObject text_completion id=cmpl-6fJfeT99n9jDjfpPzrAi7XyxTLbzV at 0x1137be540> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " eCommerce"
    }
  ],
  "created": 1675305082,
  "id": "cmpl-6fJfeT99n9jDjfpPzrAi7XyxTLbzV",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 2,
    "prompt_tokens": 383,
    "total_tokens": 385
  }
}

In [11]:
global_context = 'The website name is {0}. This website is a {1} website.'.format(
    driver.current_url,
    category_res.choices[0].text.strip()
)

global_context

'The website name is https://demo.saleor.io/default-channel/en-US/. This website is a eCommerce website.'

# Page Context

In [ ]:
driver.get('https://demo.saleor.io/dashboard/')

In [15]:
body_doc = bs(driver.find_element(By.TAG_NAME, 'body').get_attribute('innerHTML'), 'html.parser')

In [16]:
headings = []
for h in range(6):
    headings = [*headings, *list(map(lambda x: x.get_text().strip(), body_doc.find_all('h' + str(h + 1))))]

In [18]:
headings = ['Sign In']

In [19]:
local_prompt = '''{0}
We are in page {1} of the website. The headings in this page are: {2}.
The page main functionality is'''.format(
    global_context,
    driver.current_url,
    ', '.join(headings)
)

print(local_prompt)

The website name is https://demo.saleor.io/default-channel/en-US/. This website is a eCommerce website.
We are in page https://demo.saleor.io/dashboard/ of the website. The headings in this page are: Sign In.
The page main functionality is


In [20]:
local_res = openai.Completion.create(
    model="text-davinci-003",
    prompt=local_prompt,
    temperature=0,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    max_tokens=256,
    stop=['.']
)

print(local_res.choices[0].text)

 to allow users to sign in to their account


In [21]:
local_context = '''{0}. We are in page {1}. The page main functionality is {2}.'''.format(
    global_context,
    driver.current_url,
    local_res.choices[0].text.strip()
)

local_context

'The website name is https://demo.saleor.io/default-channel/en-US/. This website is a eCommerce website.. We are in page https://demo.saleor.io/dashboard/. The page main functionality is to allow users to sign in to their account.'

# Input Context

In [37]:
body_doc.find_all('form')[0]

<form><div class="MuiTypography-root jss24 MuiTypography-h3">Sign In</div><div class="MuiFormControl-root MuiTextField-root MuiFormControl-fullWidth"><label class="MuiFormLabel-root MuiInputLabel-root MuiInputLabel-formControl MuiInputLabel-animated MuiInputLabel-shrink MuiInputLabel-outlined MuiFormLabel-filled" data-shrink="true">Email address</label><div class="MuiInputBase-root MuiOutlinedInput-root MuiInputBase-fullWidth MuiInputBase-formControl"><input aria-invalid="false" autocomplete="username" class="MuiInputBase-input MuiOutlinedInput-input" data-test-id="email" name="email" spellcheck="false" type="text" value="admin@example.com"/><fieldset aria-hidden="true" class="jss34 MuiOutlinedInput-notchedOutline"><legend class="jss36 jss37"><span>Email address</span></legend></fieldset></div></div><div class="jss38"></div><div class="jss29"><div class="MuiFormControl-root MuiTextField-root MuiFormControl-fullWidth"><label class="MuiFormLabel-root MuiInputLabel-root MuiInputLabel-form

In [24]:
input_prompt = '''{0}
We want to complete the form on this page.
The input on the page has label {1}, and it is a {2} input with type {3}. Provide a value for the input:'''.format(
    local_context,
    'Password',
    'input',
    'password'
)

input_prompt

'The website name is https://demo.saleor.io/default-channel/en-US/. This website is a eCommerce website.. We are in page https://demo.saleor.io/dashboard/. The page main functionality is to allow users to sign in to their account.\nWe want to complete the form on this page.\nThe input on the page has label Password, and it is a input input with type password. Provide a value for the input:'

In [25]:
input_res = openai.Completion.create(
    model="text-davinci-003",
    prompt=input_prompt,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    max_tokens=256,
)

print(input_res.choices[0].text.strip())

MyPassword123!


# Error Context

In [34]:
sample_error = 'Your password should be all in lowercase and without numbers'

In [35]:
error_prompt = '''{0}
The input on the page has label {1}, and it is a {2} input with type {3}.
We entered the following value for input: "{4}" and we got the following error: "{5}". Provide a value for the input:'''.format(
    local_context,
    'Password',
    'input',
    'password',
    'MyPassword123!',
    sample_error
)

error_prompt


'The website name is https://demo.saleor.io/default-channel/en-US/. This website is a eCommerce website.. We are in page https://demo.saleor.io/dashboard/. The page main functionality is to allow users to sign in to their account.\nThe input on the page has label Password, and it is a input input with type password.\nWe entered the following value for input: "MyPassword123!" and we got the following error: "Your password should be all in lowercase and without numbers". Provide a value for the input:'

In [36]:
error_res = openai.Completion.create(
    model="text-davinci-003",
    prompt=error_prompt,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    max_tokens=256,
)

print(error_res.choices[0].text.strip())

mypassword
